### ElasticNet Regression
##### Amaç; hata kareler toplamını minimize eden katsayıları, bu katsayılara bir ceza uygulayarak bulmaktır.
###### ElacticNet L1(Lasso) ve L2(Ridge) yaklaşımlarını birleştirir.

#### Model

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV

In [3]:
df = pd.read_csv("Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])
y = df["Salary"]
X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis=1).astype("float64")
X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)

In [5]:
import warnings 
warnings.filterwarnings('ignore')
enet_model = ElasticNet().fit(X_train, y_train)

In [6]:
# B0 sabiti
enet_model.intercept_

-6.465955602112331

In [7]:
# B1 katsayıları
enet_model.coef_

array([ -1.86256172,   8.70489065,   5.10426375,  -2.89875799,
        -1.28642985,   5.24343682,   6.04480276,  -0.14701495,
        -0.21566628,  -0.7897201 ,   1.80813117,   0.80914508,
        -0.61262382,   0.26816203,   0.27172387,  -0.36530729,
        19.2186222 , -31.16586592,   8.98369938])

#### Predict

In [9]:
enet_model.predict(X_train)[0:10]

array([325.74706292, 776.06632333, 522.86508419, 107.64091955,
       449.03139566, 997.76095723,  99.78828622, 311.33763086,
       418.50335021, 879.9502608 ])

In [10]:
enet_model.predict(X_test)[0:10]

array([ 577.79111731,  617.33202224, 1031.39113156,  364.95861575,
        489.51894393,  300.74185842,  604.522666  ,  465.34678732,
        901.44473965,  703.20357123])

In [13]:
# test setindekş bağımsız değişkenleri kullanarak bağımlı değişkenlik tahmin etme
y_pred = enet_model.predict(X_test)

In [14]:
# Test Error
np.sqrt(mean_squared_error(y_test, y_pred))

357.1676548181246

In [15]:
r2_score(y_test, y_pred)

0.41070222469326867

#### Model Tuming

In [17]:
enet_cv_model = ElasticNetCV(cv=10).fit(X_train, y_train)

In [18]:
# Kendisinin bulduğu lambda değeri
enet_cv_model.alpha_

5230.7647364798695

In [19]:
# Bulduğu B0 sabiti
enet_cv_model.intercept_

-38.5194055839429

In [20]:
# Bulduğu B1 katsayıları
enet_cv_model.coef_

array([ 0.62845434,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        ,  0.09788752,  0.        ,
        0.27265769,  0.19270075,  0.00758665,  0.3106529 ,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        ])

##### Final Model

In [22]:
enet_tuned = ElasticNet(alpha=enet_cv_model.alpha_).fit(X_train, y_train)

In [23]:
y_pred = enet_tuned.predict(X_test)

In [26]:
# Final modeline ilişkin test seti hatası
np.sqrt(mean_squared_error(y_test, y_pred))

394.15280563218795